# MeerKAT conversion guide

In [1]:
from importlib.metadata import version
import os

try:
    import xradio

    print("XRADIO version", version("xradio"), "already installed.")
except ImportError as e:
    print(e)
    print("Installing XRADIO")

    os.system("pip install xradio")

    import xradio

    print("xradio version", version("xradio"), " installed.")

XRADIO version 0.0.36 already installed.


## Download dataset

In [2]:
import graphviper

graphviper.utils.data.download(file="small_meerkat.ms")

[2024-08-29 14:15:43,473]  WARNING  graphviper:  File exists: /home/fedemp/ws_xradio_pointing/venv_xradio_python_39/lib/python3.9/site-packages/graphviper/utils/data/.dropbox 
[2024-08-29 14:15:43,474]     INFO  graphviper:  Updating file metadata information ...  
 

 Download List     
 ────────────────── 
  small_meerkat.ms

small_meerkat.ms.zip:   0%|          | 0.00/2.28M [00:00<?, ?iB/s]

## Convert to Processing Set

In [3]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

ms_file = "small_meerkat.ms"
main_chunksize = {"frequency": 1, "time": 20}  # baseline, polarization
outfile = "small_meerkat.vis.zarr"
convert_msv2_to_processing_set(
    in_file=ms_file,
    out_file=outfile,
    parallel=False,
    overwrite=True,
    main_chunksize=main_chunksize,
)

[2024-08-29 14:15:48,387]     INFO  graphviper:  Partition scheme that will be used: ['DATA_DESC_ID', 'OBS_MODE', 'OBSERVATION_ID', 'FIELD_ID'] 
[2024-08-29 14:15:48,465]     INFO  graphviper:  Number of partitions: 3 
[2024-08-29 14:15:48,466]     INFO  graphviper:  OBSERVATION_ID [0], DDI [0], STATE [1], FIELD [0], SCAN [1] 
[2024-08-29 14:15:48,829]     INFO  graphviper:  OBSERVATION_ID [0], DDI [0], STATE [2], FIELD [1], SCAN [2 4 6] 
[2024-08-29 14:15:49,134]     INFO  graphviper:  OBSERVATION_ID [0], DDI [0], STATE [3], FIELD [2], SCAN [3 5] 


## Processing Set

In [4]:
from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set(ps_store=outfile)
ps.summary()

,name,obs_mode,shape,polarization,scan_number,spw_name,field_name,source_name,line_name,field_coords,start_frequency,end_frequency
0,small_meerkat_0,"[CALIBRATE_BANDPASS, CALIBRATE_FLUX]","(74, 6, 50, 4)","[XX, XY, YX, YY]",[1],spw_0,[J1939-6342_0],[J1939-6342_0],[],"[fk5, 19h39m25.03s, -63d42m45.60s]",3.265869e+09,3.276337e+09
2,small_meerkat_1,"[CALIBRATE_PHASE, CALIBRATE_AMPLI]","(42, 6, 50, 4)","[XX, XY, YX, YY]","[2, 4, 6]",spw_0,[J1619-8418_1],[J1619-8418_1],[],"[fk5, 16h19m33.97s, -84d18m19.10s]",3.265869e+09,3.276337e+09
1,small_meerkat_2,[TARGET],"(223, 6, 50, 4)","[XX, XY, YX, YY]","[3, 5]",spw_0,[J0358-8103_2],[J0358-8103_2],[],"[fk5, 3h58m31.50s, -81d03m45.70s]",3.265869e+09,3.276337e+09


In [5]:
ps.get(0).attrs["antenna_xds"]

<xarray.Dataset> Size: 580B
Dimensions:                (antenna_name: 3, cartesian_pos_label: 3,
                            receptor_label: 2, sky_dir_label: 2)
Coordinates:
  * antenna_name           (antenna_name) <U4 48B 'm000' 'm002' 'm063'
  * cartesian_pos_label    (cartesian_pos_label) <U1 12B 'x' 'y' 'z'
    mount                  (antenna_name) <U6 72B dask.array<chunksize=(3,), meta=np.ndarray>
    polarization_type      (antenna_name, receptor_label) <U1 24B dask.array<chunksize=(3, 2), meta=np.ndarray>
  * receptor_label         (receptor_label) <U5 40B 'pol_0' 'pol_1'
  * sky_dir_label          (sky_dir_label) <U3 24B 'ra' 'dec'
    station                (antenna_name) <U4 48B dask.array<chunksize=(3,), meta=np.ndarray>
Data variables:
    ANTENNA_DISH_DIAMETER  (antenna_name) float64 24B dask.array<chunksize=(3,), meta=np.ndarray>
    ANTENNA_FEED_OFFSET    (antenna_name, cartesian_pos_label) float64 72B dask.array<chunksize=(3, 3), meta=np.ndarray>
    ANTENNA_POSITION       (antenna_name, cartesian_pos_label) float64 72B dask.array<chunksize=(3, 3), meta=np.ndarray>
    BEAM_OFFSET            (antenna_name, receptor_label, sky_dir_label) float64 96B dask.array<chunksize=(3, 2, 2), meta=np.ndarray>
    RECEPTOR_ANGLE         (antenna_name, receptor_label) float64 48B dask.array<chunksize=(3, 2), meta=np.ndarray>
Attributes:
    overall_telescope_name:  MeerKAT
    relocatable_antennas:    False
    type:                    antenna